# Automação Web e Busca de Informações com Python

**Buscando os dados usando selenium**

In [21]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
import pandas as pd
serviço = Service(ChromeDriverManager().install())

navegador = webdriver.Chrome(service=serviço)

# Pegar a cotação do dólar
navegador.get('https://google.com.br/')
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação dólar', Keys.ENTER)
valorDolar = navegador.find_element('xpath', '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

# Pegar a cotação do euro
navegador.get('https://google.com.br/')
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação euro', Keys.ENTER)
valorEuro = navegador.find_element('xpath', '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

# Pegar a cotação do ouro
navegador.get('https://www.melhorcambio.com/ouro-hoje')
valorOuro = navegador.find_element('xpath', '//*[@id="comercial"]').get_attribute('value')
valorOuro = valorOuro.replace(',', '.')

navegador.quit()

**Atualizando a base de dados**

In [22]:
tabela = pd.read_excel('Produtos.xlsx')
tabela.loc[tabela['Moeda']=='Dólar', 'Cotação'] = float(valorDolar)
tabela.loc[tabela['Moeda']=='Euro', 'Cotação'] = float(valorEuro)
tabela.loc[tabela['Moeda']=='Ouro', 'Cotação'] = float(valorOuro)

# Atualizar o preço de compra
tabela['Preço de Compra'] = tabela['Preço Original'] * tabela['Cotação']
tabela['Preço de Venda'] = tabela['Preço de Compra'] * tabela['Margem'] 
tabela['Preço de Venda'] = tabela['Preço de Venda'].replace(',', '.')
tabela

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.0885,5088.449115,1.40,7123.828761
1,Carro Renault,4500.00,Euro,5.2215,23496.750000,2.00,46993.500000
2,Notebook Dell,899.99,Dólar,5.0885,4579.599115,1.70,7785.318495
3,IPhone,799.00,Dólar,5.0885,4065.711500,1.70,6911.709550
4,Carro Fiat,3000.00,Euro,5.2215,15664.500000,1.90,29762.550000
5,Celular Xiaomi,480.48,Dólar,5.0885,2444.922480,2.00,4889.844960
6,Joia 20g,20.00,Ouro,296.2100,5924.200000,1.15,6812.830000


**Gerar a tabela com os dados atualizados**

In [23]:
tabela.to_excel('Produtos Novos.xlsx', index=False)